In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_parquet(
    '/nvme1/scratch/walml/repos/curation-datasets/gz2_downloadable_catalog.parquet'
    # columns=useful_cols
)

In [3]:
len(df)

209239

In [4]:
df['label'].value_counts()  # -1 if could not be assigned to leaf -but not dropped

 0    78460
 1    54997
 4    26132
 2    19734
-1    11762
 5     9246
 3     4613
 6     4295
Name: label, dtype: int64

In [5]:
df.head()

,dr7objid,png_loc,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,disk-edge-on_yes,disk-edge-on_no,bar_yes,bar_no,has-spiral-arms_yes,...,bulge-shape_semantic,spiral-winding_semantic,spiral-arm-count_semantic,temp_semantic_label,summary,leaf_prob,sample,asset_id,filename,label
0,587732591714893851,/raid/scratch/walml/galaxy_zoo/gz2/png/587732/...,0,42,3,2,40,1,39,40,...,-,spiral-winding_tight,spiral-arm-count_more-than-4,smooth-or-featured_featured-or-disk - disk-edg...,unbarred_spiral,0.866667,original,58957,58957.jpg,4
1,588009368545984617,/raid/scratch/walml/galaxy_zoo/gz2/png/588009/...,1,41,0,0,41,4,37,41,...,-,spiral-winding_tight,spiral-arm-count_more-than-4,smooth-or-featured_featured-or-disk - disk-edg...,unbarred_spiral,0.880952,original,193641,193641.jpg,4
2,587732484359913515,/raid/scratch/walml/galaxy_zoo/gz2/png/587732/...,28,5,3,0,5,2,3,0,...,-,-,-,smooth-or-featured_smooth how-rounded_in-betwe...,smooth_inbetween,0.638889,original,55934,55934.jpg,0
3,587741723357282317,/raid/scratch/walml/galaxy_zoo/gz2/png/587741/...,1,27,0,1,26,1,25,25,...,-,spiral-winding_tight,deadlock,smooth-or-featured_featured-or-disk - disk-edg...,unbarred_spiral,0.858516,original,158501,158501.jpg,4
4,587738410866966577,/raid/scratch/walml/galaxy_zoo/gz2/png/587738/...,33,8,2,1,7,0,7,0,...,-,-,-,smooth-or-featured_smooth how-rounded_round - - -,smooth_round,0.418605,original,110939,110939.jpg,1


In [6]:
df['id_str'] = df['dr7objid']
del df['asset_id']
del df['dr7objid']

In [7]:
# convert to ortho labels
from zoobot.shared import label_metadata

current_labels = label_metadata.gz2_label_cols
ortho_labels = label_metadata.gz2_ortho_label_cols

count_renamer = dict(zip(current_labels, ortho_labels))

fraction_renamer = dict(zip([x + '_fraction' for x in current_labels], [x + '_fraction' for x in ortho_labels]))

renamer = count_renamer
renamer.update(fraction_renamer)
# renamer

df = df.rename(columns=renamer)

In [9]:
temp = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz2/images/' + df['filename']

import os
assert os.path.isfile(temp[0])

is_file = [os.path.isfile(loc) for loc in temp]
np.mean(is_file), len(is_file) -  np.sum(is_file)

(1.0, 0)

In [10]:
from PIL import Image

for loc in temp:
    if not Image.open(loc).size == (424, 424):
        print(loc)
        break

In [13]:
train_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz2/gz2_train_catalog.parquet'
test_catalog_loc = '/home/walml/repos/pytorch-galaxy-datasets/roots/gz2/gz2_test_catalog.parquet'

# zip not tar.gz as directly from Zenodo
images_loc = '/nvme1/scratch/walml/repos/pytorch-galaxy-datasets/roots/gz2/images_gz2.zip'

In [14]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

train_catalog.to_parquet(train_catalog_loc, index=False)
test_catalog.to_parquet(test_catalog_loc, index=False)

In [15]:
import hashlib

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    images_loc  # compressed from images above, takes 5 secs
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

f489c9ec7dcf8d99f728bd00ee00b1d0
8b2d74c93d35f86cc34f1d058b3b220b
bc647032d31e50c798770cf4430525c7
